In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
from context import Context
from contracts.farm_contract import FarmContract
from contracts.pair_contract import PairContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.position_creator_contract import PositionCreatorContract
from contracts.fees_collector_contract import FeesCollectorContract
from utils.utils_generic import log_warning, log_step_pass, log_step_fail
context = Context()
fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]
log_step_pass(f"Fees collector contract: {fees_collector_contract.address}")

In [ ]:
MOAUSDC_POOL_ADDRESS = "drt1qqqqqqqqqqqqqpgqv0c22xku4vd5ugfh0u53tqufmdkqx5nv2jps6s8mmc"
KWAKREWA_POOL_ADDRESS = "drt1qqqqqqqqqqqqqpgqthhxe87ulznhhm72jekexj9rwqzu3zsv2jps6mwq7z"

moausdc_pair_contract = PairContract.load_contract_by_address(MOAUSDC_POOL_ADDRESS)
kwakrewa_pair_contract = PairContract.load_contract_by_address(KWAKREWA_POOL_ADDRESS)

log_warning(f'Pair contract: {moausdc_pair_contract.lpToken} {moausdc_pair_contract.address}')
log_warning(f'Pair contract: {kwakrewa_pair_contract.lpToken} {kwakrewa_pair_contract.address}')

energy_factory: SimpleLockEnergyContract
energy_factory = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

moausdc_farm_contract = FarmContract.load_contract_by_address("drt1qqqqqqqqqqqqqpgqzfy62c66hjuu9lm2c9mr4gcver9lmnydkp2skzc6r2")
foxsy_farm_contract = FarmContract.load_contract_by_address("drt1qqqqqqqqqqqqqpgqu288dj3qgrelujal5wynsv5zrf7u0hwlkp2sydsjsq")
kwakrewa_farm_contract = FarmContract.load_contract_by_address("drt1qqqqqqqqqqqqqpgqgg42sl557vh7rwt600w76pe48u64qngmkp2szmvq2c")
log_warning(f'New MOAUSC Farm contract: {moausdc_farm_contract.farmToken}')
log_warning(f'New FOXSY Farm contract: {foxsy_farm_contract.farmToken}')
log_warning(f'New KWAKREWA Farm contract: {kwakrewa_farm_contract.farmToken}')

position_creator_contract: PositionCreatorContract
position_creator_contract = context.get_contracts(config.POSITION_CREATOR)[0]

Change owners

In [ ]:
new_owner = "drt1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsaws9xq"

In [ ]:
moausdc_farm_contract.change_owner_address(context.deployer_account, context.network_provider.proxy, new_owner)

In [ ]:
kwakrewa_farm_contract.change_owner_address(context.deployer_account, context.network_provider.proxy, new_owner)

Take ownership

In [ ]:
old_owner = "drt19uysnnzwnzm3klq8qzv7zprw2d3yj38hx0ac9cuqvl0j2jr5kp2s5rmc8j"

In [ ]:
moausdc_farm_contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, old_owner)

In [ ]:
kwakrewa_farm_contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, old_owner)

REWAMOA farm config

In [ ]:
moausdc_pair_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, moausdc_farm_contract.address)

In [ ]:
energy_factory.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, moausdc_farm_contract.address)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [moausdc_farm_contract.address])

KWAKREWA farm config

In [ ]:
kwakrewa_pair_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, kwakrewa_farm_contract.address)

In [ ]:
energy_factory.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, kwakrewa_farm_contract.address)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [kwakrewa_farm_contract.address])

Position creator whitelist

In [ ]:
moausdc_farm_contract.add_contract_to_whitelist(context.deployer_account, context.network_provider.proxy, position_creator_contract.address)

In [ ]:
kwakrewa_farm_contract.add_contract_to_whitelist(context.deployer_account, context.network_provider.proxy, position_creator_contract.address)

Farm rewards distribution

In [ ]:
import json

rewards_config_file = "emission_rewards_24_12_06.json"
log_warning(f"Reading rewards config from {rewards_config_file}")

# load the rewards json file
with open(rewards_config_file) as f:
    emission_rewards = json.load(f)

In [ ]:
from tools.runners.farm_runner import get_farm_addresses_from_chain
from utils.utils_generic import get_continue_confirmation
from utils.utils_chain import get_account_key, string_to_hex

# set the rewards
farm_addresses = get_farm_addresses_from_chain("v2")
log_step_pass(f"Retrieved {len(farm_addresses)} farm addresses.")
if get_continue_confirmation():
    for farm_address in farm_addresses:
        farm_contract: FarmContract
        farm_contract = FarmContract.load_contract_by_address(farm_address)
        log_step_pass(f"Setting rewards for farm {farm_contract.farmToken} {farm_contract.address}")
        
        if farm_contract.farmToken not in emission_rewards:
            log_step_fail(f"Farm token {farm_contract.farmToken} not found in rewards config. Skipping.")
            continue

        new_rewards = emission_rewards[farm_contract.farmToken] * 10**18
        log_warning(f"New rewards: {new_rewards:,}")

        if not new_rewards:
            new_rewards = 1     # can't set 0 rpb
            # check if rewards producing is stopped, otherwise offer to stop it
            rewards_enabled = get_account_key(farm_address, string_to_hex("produce_rewards_enabled"), context.network_provider.proxy)
            if rewards_enabled == "01":
                log_warning(f"Produce rewards enabled for farm {farm_contract.farmToken}. Want to stop?")
                if get_continue_confirmation():
                    tx_hash = farm_contract.end_produce_rewards(context.deployer_account, context.network_provider.proxy)
                    if not context.network_provider.check_simple_tx_status(tx_hash, f"stop rewards for: {farm_address}"):
                        if not get_continue_confirmation():
                            break
                
        # if not get_continue_confirmation():
        #      break
        tx_hash = farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, new_rewards)
        if not context.network_provider.check_simple_tx_status(tx_hash, f"set rewards per block for: {farm_address}"):
                if not get_continue_confirmation():
                    break

Fees collector rewards distribution

In [ ]:
new_rewards = emission_rewards["FEESCOLLECTOR"] * 10**18
log_warning(f"New fees collector rewards: {new_rewards:,}")
tx_hash = fees_collector_contract.set_locked_tokens_per_block(context.deployer_account, context.network_provider.proxy, new_rewards)
context.network_provider.check_simple_tx_status(tx_hash, f"set fees collector rewards per block")

Start new farms

In [ ]:
moausdc_farm_contract.resume(context.deployer_account, context.network_provider.proxy)

In [ ]:
foxsy_farm_contract.resume(context.deployer_account, context.network_provider.proxy)

In [ ]:
kwakrewa_farm_contract.resume(context.deployer_account, context.network_provider.proxy)

Set rewards per block

In [ ]:
moausdc_farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 10400 * 10**18)

In [ ]:
foxsy_farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 2600 * 10**18)

In [ ]:
kwakrewa_farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 1300 * 10**18)

Set rewards active for new farms

In [ ]:
moausdc_farm_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

In [ ]:
foxsy_farm_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

In [ ]:
kwakrewa_farm_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)